# Ajuste de un histograma con [curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)

## Datos

In [ ]:
import pandas as pd
data = pd.read_csv("histogram.csv")
data

La función `curve_fit` ajusta los datos con cuadrados mínimos solamente. Por lo tanto asume que la variable independiente $Y$ es normal. Para calcular la función de costo normal es necesario conocer la desviación estándar de $Y$. Como $Y$ es una variable poissoniana, aproximamos su desviación estándar como la raíz cuadrada del valor medido $y$.

In [ ]:
import numpy as np
data['ysigma'] = np.sqrt(data['y'])
data

La función de costo normal $ \left( \frac{y-\mu}{\sigma} \right)^2$ requiere $\sigma \ne 0$. 
Por lo tanto los bines sin entradas ($y=0$) no pueden ser ajustados data que para éstos $\sigma = \sqrt{y} = 0$.
Sacamos estos bines de los datos usados para el ajuste.

In [ ]:
fit_data = data.loc[data['y']>0]
fit_data

Vemos que el bin $x=-2.9$ es rechazado de esta forma.

## Modelo

El modelo es una PDF normal escalada por una constante de normalización.  
Parámetros:
- Constante de normalización
- Media $\mu$
- Desviación estándar $\sigma$

In [ ]:
from scipy.stats import norm
def fit_model(x, par0, par1, par2):
    return par0 * norm.pdf(x, loc=par1, scale=par2)

## Minimización

Estimamos los errores en la variable dependiente $y$ considerando que sigue una distribución de Poisson

In [ ]:
import scipy
par_est, cova = scipy.optimize.curve_fit(f=fit_model, 
                                            xdata=fit_data['x'], 
                                            ydata=fit_data['y'], 
                                            sigma=fit_data['ysigma'], 
                                            absolute_sigma=True)

## Parámetros

In [ ]:
import numpy as np
error = np.sqrt(np.diag(cova))
print(f"Normalization = {par_est[0]:.2f} ± {error[0]:.2f}")
print(f"Mean = {par_est[1]:.3f} ± {error[1]:.3f}")
print(f"Standard deviation = {par_est[2]:.3f} ± {error[2]:.3f}")

Los estimadores de `curve_fit` son diferentes a los calculados con una función de costo de Poisson, la distribución exacta de la variable dependiente $y$. Estas diferencias son pocas si el número de entradas en el histograma es grande. Sin embargo para histogramas con pocas entradas es incorrecto usar cuadrados mínimos. En este segundo caso para ajustar datos se debe usar alguna opción distinta a `curve_fit`. 